In [6]:
import urllib
import urllib.parse
#get - method1
# response = urllib.request.urlopen('https://www.baidu.com')
# print(response.read().decode('utf-8')) #对获取的网页进行utf-8解码
#post - method2 httpbin.org
data = bytes(urllib.parse.urlencode({"hello":"world"}),encoding='utf-8')
postresponse = urllib.request.urlopen('http://www.httpbin.org/post',data=data)
print(postresponse.read().decode('utf-8'))

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "hello": "world"
  }, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Content-Length": "11", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "www.httpbin.org", 
    "User-Agent": "Python-urllib/3.8", 
    "X-Amzn-Trace-Id": "Root=1-60030772-655f3c2b682fea332dac727c"
  }, 
  "json": null, 
  "origin": "219.78.103.76", 
  "url": "http://www.httpbin.org/post"
}



In [14]:
import urllib
# try:
#     getresponse = urllib.request.urlopen('http://www.httpbin.org/get',timeout=0.01)
#     print(getresponse.read().decode('utf-8'))
# except urllib.error.URLError as e:
#     print("time out!")

In [16]:
getresponse = urllib.request.urlopen('http://www.httpbin.org/get')
print(getresponse.status)

200


In [18]:
# response = urllib.request.urlopen("http://douban.com")
# print(reponse.status)
#error 418 证明对方知道你是一个查重

In [24]:
response = urllib.request.urlopen("http://www.baidu.com")
print(response.getheaders())

[('Connection', 'close'), ('Transfer-Encoding', 'chunked'), ('Bdpagetype', '1'), ('Bdqid', '0x81457e740010454c'), ('Cache-Control', 'private'), ('Content-Type', 'text/html;charset=utf-8'), ('Date', 'Sat, 16 Jan 2021 15:54:26 GMT'), ('Expires', 'Sat, 16 Jan 2021 15:54:13 GMT'), ('P3p', 'CP=" OTI DSP COR IVA OUR IND COM "'), ('P3p', 'CP=" OTI DSP COR IVA OUR IND COM "'), ('Server', 'BWS/1.1'), ('Set-Cookie', 'BAIDUID=37101F11D0D5996E9E2BAE2CE635C983:FG=1; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2147483647; path=/; domain=.baidu.com'), ('Set-Cookie', 'BIDUPSID=37101F11D0D5996E9E2BAE2CE635C983; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2147483647; path=/; domain=.baidu.com'), ('Set-Cookie', 'PSTM=1610812466; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2147483647; path=/; domain=.baidu.com'), ('Set-Cookie', 'BAIDUID=37101F11D0D5996E3BDCAA15737C7DDD:FG=1; max-age=31536000; expires=Sun, 16-Jan-22 15:54:26 GMT; domain=.baidu.com; path=/; version=1; comment=bd'), ('Set-Cookie', 'BDSVRT